In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

import matplotlib.colors as colors
import matplotlib.cbook as cbook

from IPython.display import display

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from eis.EISDataIO import eis_dataframe_from_csv

from os import path
import logging

2022-03-21 06:08:23.160579: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-21 06:08:23.160631: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# loading training data

# if you are on a windows machine un-comment the following line to get the path to training data
# here = !echo %cd%

# if you are on a mac/ unix machine un-comment the following line to get the path to training data
here = !pwd

train_data_path = path.join(path.dirname(here[0]), "train_data.csv")
eis_data = eis_dataframe_from_csv(train_data_path)

In [3]:
def complex_parts(cmp: pd.Series) -> tuple[pd.Series, pd.Series]:
    real_part= cmp.apply(np.real)
    imag_part= cmp.apply(np.imag)
    return (real_part, imag_part)

eis_data["Z_real"], eis_data["Z_imag"] = complex_parts(eis_data.Z)
display(eis_data)

,freq,Z,Circuit,Parameters,Z_real,Z_imag
0,"[0.1, 0.148398179, 0.220220195, 0.326802759, 0...","[(309.82561192-156.06088233j), (267.46983919-1...",L-R-RCPE-RCPE-RCPE,"L1: 2.94e-08, R1: 4.51e+00, R2: 5.19e-02, CPE1...","[309.82561192, 267.46983919, 229.38061493, 196...","[-156.06088233, -146.21033646, -133.4855237, -..."
1,"[1.0, 1.34339933, 1.80472177, 2.42446202, 3.25...","[(344.50700012-0.87321496j), (344.36191597-0.9...",RC-RC-RCPE-RCPE,"R1: 2.08e+02, R2: 2.50e+01, R3: 9.57e+01, R4: ...","[344.50700012, 344.36191597, 344.19647198, 344...","[-0.87321496, -0.99738889, -1.13909869, -1.300..."
2,"[1.0, 1.26360956, 1.59670912, 2.01761691, 2.54...","[(3080.15920083-80.84202473j), (3071.83539583-...",L-R-RCPE-RCPE-RCPE,"L1: 3.35e-08, R1: 6.95e+01, R2: 7.49e+01, CPE1...","[3080.15920083, 3071.83539583, 3061.93812951, ...","[-80.84202473, -94.50641483, -110.36642266, -1..."
3,"[10.0, 13.4990445, 18.2224203, 24.5985264, 33....","[(930.93345951-0.0068507146j), (930.93327153-0...",L-R-RCPE,"L1: 8.43e-07, R1: 9.06e+01, R2: 8.40e+02, CPE1...","[930.93345951, 930.93327153, 930.93302176, 930...","[-0.0068507146, -0.00910185162, -0.0120927041,..."
4,"[0.01, 0.0148907532, 0.0221734532, 0.033017942...","[(405.07355219-0.0149508921j), (405.07110253-0...",RCPE-RCPE-RCPE,"R1: 1.03e+01, R2: 6.71e-01, R3: 3.94e+02, CPE1...","[405.07355219, 405.07110253, 405.06786553, 405...","[-0.0149508921, -0.0197551995, -0.026103215, -..."
...,...,...,...,...,...,...
7457,"[10.0, 13.4596032, 18.1160919, 24.383541, 32.8...","[(4953.31225754-4814.61138816j), (4269.4031861...",RCPE-RCPE,"R1: 2.47e+01, R2: 2.95e+05, CPE1_t: 9.41e-01, ...","[4953.31225754, 4269.40318615, 3680.28615178, ...","[-4814.61138816, -4165.41666719, -3601.4380483..."
7458,"[10.0, 12.6485522, 15.9985872, 20.2358965, 25....","[(73538.51000223-1776.24017654j), (73413.60429...",L-R-RCPE-RCPE-RCPE,"L1: 1.05e-08, R1: 2.81e+01, R2: 6.40e+01, CPE1...","[73538.51000223, 73413.60429656, 73304.9395675...","[-1776.24017654, -1592.10621049, -1461.6962630..."
7459,"[1.0, 1.34453288, 1.80776868, 2.43060443, 3.26...","[(82.1093149-51.12982215j), (72.40330299-47.29...",RCPE-RCPE-RCPE,"R1: 4.78e+00, R2: 8.77e-01, R3: 2.85e+02, CPE1...","[82.1093149, 72.40330299, 63.66501847, 55.8809...","[-51.12982215, -47.29789889, -43.31241432, -39..."
7460,"[0.01, 0.0134051824, 0.0179698915, 0.024088967...","[(32.37740171-5.15196774e-05j), (32.37740063-6...",RCPE-RCPE,"R1: 5.65e+00, R2: 2.67e+01, CPE1_t: 9.21e-01, ...","[32.37740171, 32.37740063, 32.3773992, 32.3773...","[-5.15196774e-05, -6.82381469e-05, -9.03818659..."


In [4]:
def drange(cmp: pd.Series) -> tuple[pd.Series, pd.Series]:
    min_= cmp.apply(np.min)
    max_= cmp.apply(np.max)
    return (min_, max_)

eis_data["freq_min"], eis_data["freq_max"] = drange(eis_data.freq)
eis_data["Z_real_min"], eis_data["Z_real_max"] = drange(eis_data.Z_real)
eis_data["Z_imag_min"], eis_data["Z_imag_max"] = drange(eis_data.Z_imag)
display(eis_data)

,freq,Z,Circuit,Parameters,Z_real,Z_imag,freq_min,freq_max,Z_real_min,Z_real_max,Z_imag_min,Z_imag_max
0,"[0.1, 0.148398179, 0.220220195, 0.326802759, 0...","[(309.82561192-156.06088233j), (267.46983919-1...",L-R-RCPE-RCPE-RCPE,"L1: 2.94e-08, R1: 4.51e+00, R2: 5.19e-02, CPE1...","[309.82561192, 267.46983919, 229.38061493, 196...","[-156.06088233, -146.21033646, -133.4855237, -...",0.10,100000.000,4.802034,309.825612,-156.060882,-0.369196
1,"[1.0, 1.34339933, 1.80472177, 2.42446202, 3.25...","[(344.50700012-0.87321496j), (344.36191597-0.9...",RC-RC-RCPE-RCPE,"R1: 2.08e+02, R2: 2.50e+01, R3: 9.57e+01, R4: ...","[344.50700012, 344.36191597, 344.19647198, 344...","[-0.87321496, -0.99738889, -1.13909869, -1.300...",1.00,100000.000,230.202174,344.507000,-43.334151,-0.873215
2,"[1.0, 1.26360956, 1.59670912, 2.01761691, 2.54...","[(3080.15920083-80.84202473j), (3071.83539583-...",L-R-RCPE-RCPE-RCPE,"L1: 3.35e-08, R1: 6.95e+01, R2: 7.49e+01, CPE1...","[3080.15920083, 3071.83539583, 3061.93812951, ...","[-80.84202473, -94.50641483, -110.36642266, -1...",1.00,1995262.310,146.222822,3080.159201,-860.278825,-4.597264
3,"[10.0, 13.4990445, 18.2224203, 24.5985264, 33....","[(930.93345951-0.0068507146j), (930.93327153-0...",L-R-RCPE,"L1: 8.43e-07, R1: 9.06e+01, R2: 8.40e+02, CPE1...","[930.93345951, 930.93327153, 930.93302176, 930...","[-0.0068507146, -0.00910185162, -0.0120927041,...",10.00,199526.231,916.677733,930.933460,-78.835912,-0.006851
4,"[0.01, 0.0148907532, 0.0221734532, 0.033017942...","[(405.07355219-0.0149508921j), (405.07110253-0...",RCPE-RCPE-RCPE,"R1: 1.03e+01, R2: 6.71e-01, R3: 3.94e+02, CPE1...","[405.07355219, 405.07110253, 405.06786553, 405...","[-0.0149508921, -0.0197551995, -0.026103215, -...",0.01,1995262.310,17.754399,405.073552,-120.693060,-0.014951
...,...,...,...,...,...,...,...,...,...,...,...,...
7457,"[10.0, 13.4596032, 18.1160919, 24.383541, 32.8...","[(4953.31225754-4814.61138816j), (4269.4031861...",RCPE-RCPE,"R1: 2.47e+01, R2: 2.95e+05, CPE1_t: 9.41e-01, ...","[4953.31225754, 4269.40318615, 3680.28615178, ...","[-4814.61138816, -4165.41666719, -3601.4380483...",10.00,100000.000,72.555800,4953.312258,-4814.611388,-48.358524
7458,"[10.0, 12.6485522, 15.9985872, 20.2358965, 25....","[(73538.51000223-1776.24017654j), (73413.60429...",L-R-RCPE-RCPE-RCPE,"L1: 1.05e-08, R1: 2.81e+01, R2: 6.40e+01, CPE1...","[73538.51000223, 73413.60429656, 73304.9395675...","[-1776.24017654, -1592.10621049, -1461.6962630...",10.00,1000000.000,952.937485,73538.510002,-25026.297801,-1347.863875
7459,"[1.0, 1.34453288, 1.80776868, 2.43060443, 3.26...","[(82.1093149-51.12982215j), (72.40330299-47.29...",RCPE-RCPE-RCPE,"R1: 4.78e+00, R2: 8.77e-01, R3: 2.85e+02, CPE1...","[82.1093149, 72.40330299, 63.66501847, 55.8809...","[-51.12982215, -47.29789889, -43.31241432, -39...",1.00,1995262.310,5.685886,82.109315,-51.129822,-0.082206
7460,"[0.01, 0.0134051824, 0.0179698915, 0.024088967...","[(32.37740171-5.15196774e-05j), (32.37740063-6...",RCPE-RCPE,"R1: 5.65e+00, R2: 2.67e+01, CPE1_t: 9.21e-01, ...","[32.37740171, 32.37740063, 32.3773992, 32.3773...","[-5.15196774e-05, -6.82381469e-05, -9.03818659...",0.01,100000.000,6.077507,32.377402,-12.515715,-0.000052


In [5]:
def minmaxer(cmp: pd.Series, min_, max_) -> pd.Series:
    minmaxed = (cmp-min_)/(max_-min_)
    return minmaxed

Z_real_min = eis_data["Z_real_min"].min()
Z_real_max = eis_data["Z_real_max"].max()
Z_imag_min = eis_data["Z_imag_min"].min()
Z_imag_max = eis_data["Z_imag_max"].max()
eis_data["Z_real_minmaxed"] = minmaxer(eis_data.Z_real,Z_real_min,Z_real_max)
eis_data["Z_imag_minmaxed"] = minmaxer(eis_data.Z_imag,Z_imag_min,Z_imag_max)
display(eis_data)

,freq,Z,Circuit,Parameters,Z_real,Z_imag,freq_min,freq_max,Z_real_min,Z_real_max,Z_imag_min,Z_imag_max,Z_real_minmaxed,Z_imag_minmaxed
0,"[0.1, 0.148398179, 0.220220195, 0.326802759, 0...","[(309.82561192-156.06088233j), (267.46983919-1...",L-R-RCPE-RCPE-RCPE,"L1: 2.94e-08, R1: 4.51e+00, R2: 5.19e-02, CPE1...","[309.82561192, 267.46983919, 229.38061493, 196...","[-156.06088233, -146.21033646, -133.4855237, -...",0.10,100000.000,4.802034,309.825612,-156.060882,-0.369196,"[0.0024010756884318006, 0.002387371182141883, ...","[0.9964493638297757, 0.9964559871114435, 0.996..."
1,"[1.0, 1.34339933, 1.80472177, 2.42446202, 3.25...","[(344.50700012-0.87321496j), (344.36191597-0.9...",RC-RC-RCPE-RCPE,"R1: 2.08e+02, R2: 2.50e+01, R3: 9.57e+01, R4: ...","[344.50700012, 344.36191597, 344.19647198, 344...","[-0.87321496, -0.99738889, -1.13909869, -1.300...",1.00,100000.000,230.202174,344.507000,-43.334151,-0.873215,"[0.0024122970939710123, 0.002412250150980291, ...","[0.9965537084667062, 0.9965536249749967, 0.996..."
2,"[1.0, 1.26360956, 1.59670912, 2.01761691, 2.54...","[(3080.15920083-80.84202473j), (3071.83539583-...",L-R-RCPE-RCPE-RCPE,"L1: 3.35e-08, R1: 6.95e+01, R2: 7.49e+01, CPE1...","[3080.15920083, 3071.83539583, 3061.93812951, ...","[-80.84202473, -94.50641483, -110.36642266, -1...",1.00,1995262.310,146.222822,3080.159201,-860.278825,-4.597264,"[0.003297436482895553, 0.003294743257612999, 0...","[0.9964999392686594, 0.99649075164539, 0.99648..."
3,"[10.0, 13.4990445, 18.2224203, 24.5985264, 33....","[(930.93345951-0.0068507146j), (930.93327153-0...",L-R-RCPE,"L1: 8.43e-07, R1: 9.06e+01, R2: 8.40e+02, CPE1...","[930.93345951, 930.93327153, 930.93302176, 930...","[-0.0068507146, -0.00910185162, -0.0120927041,...",10.00,199526.231,916.677733,930.933460,-78.835912,-0.006851,"[0.0026020394734222194, 0.0026020394125999763,...","[0.9965542909902029, 0.9965542894765899, 0.996..."
4,"[0.01, 0.0148907532, 0.0221734532, 0.033017942...","[(405.07355219-0.0149508921j), (405.07110253-0...",RCPE-RCPE-RCPE,"R1: 1.03e+01, R2: 6.71e-01, R3: 3.94e+02, CPE1...","[405.07355219, 405.07110253, 405.06786553, 405...","[-0.0149508921, -0.0197551995, -0.026103215, -...",0.01,1995262.310,17.754399,405.073552,-120.693060,-0.014951,"[0.0024318938251909356, 0.002431893032586322, ...","[0.9965542855438289, 0.9965542823135225, 0.996..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7457,"[10.0, 13.4596032, 18.1160919, 24.383541, 32.8...","[(4953.31225754-4814.61138816j), (4269.4031861...",RCPE-RCPE,"R1: 2.47e+01, R2: 2.95e+05, CPE1_t: 9.41e-01, ...","[4953.31225754, 4269.40318615, 3680.28615178, ...","[-4814.61138816, -4165.41666719, -3601.4380483...",10.00,100000.000,72.555800,4953.312258,-4814.611388,-48.358524,"[0.003903508245800251, 0.003682224685933619, 0...","[0.9933170610547427, 0.9937535647319029, 0.994..."
7458,"[10.0, 12.6485522, 15.9985872, 20.2358965, 25....","[(73538.51000223-1776.24017654j), (73413.60429...",L-R-RCPE-RCPE-RCPE,"L1: 1.05e-08, R1: 2.81e+01, R2: 6.40e+01, CPE1...","[73538.51000223, 73413.60429656, 73304.9395675...","[-1776.24017654, -1592.10621049, -1461.6962630...",10.00,1000000.000,952.937485,73538.510002,-25026.297801,-1347.863875,"[0.02609472831582521, 0.026054314201793743, 0....","[0.9953599923411647, 0.9954837998070524, 0.995..."
7459,"[1.0, 1.34453288, 1.80776868, 2.43060443, 3.26...","[(82.1093149-51.12982215j), (72.40330299-47.29...",RCPE-RCPE-RCPE,"R1: 4.78e+00, R2: 8.77e-01, R3: 2.85e+02, CPE1...","[82.1093149, 72.40330299, 63.66501847, 55.8809...","[-51.12982215, -47.29789889, -43.31241432, -39...",1.00,1995262.310,5.685886,82.109315,-51.129822,-0.082206,"[0.002327396489029139, 0.0023242560410406555, ...","[0.9965199170738753, 0.9965224935714031, 0.996..."
7460,"[0.01, 0.0134051824, 0.0179698915, 0.024088967...","[(32.37740171-5.15196774e-05j), (32.37740063-6...",RCPE-RCPE,"R1: 5.65e+00, R2: 2.67e+01, CPE1_t: 9.21e-01, ...","[32.37740171, 32.37740063, 32.3773992, 32.3773...","[-5.15196774e-05, -6.82381469e-05,

In [6]:
from typing import List
def input_prepro(cmp: pd.DataFrame, cols: List) -> tuple[pd.Series, pd.Series]:
    res_col = []
    mask_col = []
    n_cols = len(cols)
    for idx, r in cmp.iterrows():
        len_data=len(r[cols[0]])
        res = np.zeros([83,n_cols])
        mask = np.zeros([83,1])
        mask[0:len_data]=1
        
        for i, c in enumerate(cols):
            res[0:len_data,i]=r[c]
            
        res_col.append(res)
        mask_col.append(mask)
    
    return (pd.Series(res_col), pd.Series(mask_col))

In [7]:
eis_data["input_data"],eis_data["input_mask"] = input_prepro(eis_data, ["freq", "Z_real", "Z_imag"])

# display(eis_data)
display(eis_data["input_data"].to_numpy())

array([array([[ 1.00000000e-01,  3.09825612e+02, -1.56060882e+02],
              [ 1.48398179e-01,  2.67469839e+02, -1.46210336e+02],
              [ 2.20220195e-01,  2.29380615e+02, -1.33485524e+02],
              [ 3.26802759e-01,  1.96168815e+02, -1.19162289e+02],
              [ 4.84969343e-01,  1.67918745e+02, -1.04412341e+02],
              [ 7.19685673e-01,  1.44329597e+02, -9.01633181e+01],
              [ 1.06800043e+00,  1.24873298e+02, -7.70553122e+01],
              [ 1.58489319e+00,  1.08924496e+02, -6.54647663e+01],
              [ 2.35195264e+00,  9.58472242e+01, -5.55599618e+01],
              [ 3.49025488e+00,  8.50418415e+01, -4.73606455e+01],
              [ 5.17947468e+00,  7.59639363e+01, -4.07860036e+01],
              [ 7.68624610e+00,  6.81284907e+01, -3.56845299e+01],
              [ 1.14062492e+01,  6.11119693e+01, -3.18459963e+01],
              [ 1.69266662e+01,  5.45635954e+01, -2.90020142e+01],
              [ 2.51188643e+01,  4.82324903e+01, -2.68293146e+

In [25]:
# Utility for our sequence model.

def get_sequence_model(max_seq_len: int,N_features: int,N_classes: int ):
    #Max sequence lenght: 83, Number of features: 3

    eis_features_input = keras.Input((max_seq_len, N_features))
    mask_input = keras.Input((max_seq_len,), dtype="bool")
    x = keras.layers.LSTM(16, return_sequences=True)(
        eis_features_input, mask=mask_input
    )

    x = keras.layers.LSTM(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)

    # Number of posible classes at the output
    output = keras.layers.Dense(N_classes, activation="softmax")(x)
    rnn_model = keras.Model([eis_features_input, mask_input], output)
    rnn_model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )

    return rnn_model

In [26]:
mymodel = get_sequence_model(83, 3, 9)

In [10]:
myohe = LabelBinarizer()
a = myohe.fit_transform(eis_data["Circuit"])

In [ ]:
h = mymodel.fit(
    x=(
        np.asarray(eis_data["input_data"]).astype('float32'),
        np.asarray(eis_data["input_mask"]).astype('float32')
    ),
    y=myohe.fit_transform(eis_data["Circuit"]).astype('float32'),
    batch_size=2,
    epochs=2,
    verbose=2
)
    

In [21]:
np.random.randn(len(eis_data["Circuit"]), 83, 3).astype('float32').shape

(7462, 83, 3)

In [22]:
np.random.randint(low=0, high=2, size=(len(eis_data["Circuit"]), 83, 1)).astype('float32').shape

(7462, 83, 1)

In [23]:
myohe.fit_transform(eis_data["Circuit"]).astype('float32').shape

(7462, 9)

In [ ]:
h = mymodel.fit(
    x=(
        np.random.randn(len(eis_data["Circuit"]), 83, 3).astype('float32'),
        np.random.randint(low=0, high=2, size=(len(eis_data["Circuit"]), 83, 1)).astype('float32')
    ),
    y=myohe.fit_transform(eis_data["Circuit"]).astype('float32'),
    batch_size=18,
    epochs=2,
    verbose=2
)
    

ERROR! Session/line number was not unique in database. History logging moved to new session 16
Epoch 1/2


In [11]:
h

NameError: name 'h' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 15
